# MICOM medium for VMH high fiber diet

Here we will build up the environmental conditions used for modeling. We will start from the metabolite abundances obtained from the VMH diet designer and basically go through the following step.

1. Convert to fluxes and adjust very low abundant compounds.
2. Dilute metabolites absorbed in the small intestine.
3. Add primary bile acids and mucin cores.
4. Add in missing components to allow for at least slow growth for all known taxa residing in the human gut.

That should leave us with a set of usable media for all later simulation steps.

But first let us inspect the actual diet data we got. For that we will read the diet data, rearrange it a bit and add in annotations.

In [1]:
import pandas as pd


diet = pd.read_csv("../data/vmh_eu_average.tsv", sep="\t").iloc[:, 0:2]
diet.columns = ["reaction", "flux"]
annotations = pd.read_csv("../data/agora_metabolites.csv")

diet = diet.rename(columns={diet.columns[0]: "reaction"})
diet["metabolite"] = diet.reaction.str.replace("^EX_", "", regex=True).str.replace("\\[e\\]|\\(e\\)", "", regex=True)
diet.loc[diet.metabolite == "4hpro", "metabolite"] = "4hpro_LT"  # fix name for hydroxyproline
diet.loc[diet.flux == 0, "flux"] = 1e-4  # bug in VMH designer where everything <1e-4 gets truncated to 0

diet

,reaction,flux,metabolite
0,EX_etoh[e],234.434016,etoh
1,EX_h2o[e],165892.342500,h2o
2,EX_caro[e],0.003586,caro
3,EX_retinol[e],3.002252,retinol
4,EX_thm[e],5.407858,thm
...,...,...,...
86,EX_fol[e],0.000135,fol
87,EX_hdcea[e],7.120473,hdcea
88,EX_strch1[e],33.037000,strch1
89,EX_i[e],0.002012,i


## Adjust for intestinal adsorption

To achieve this we will load the Recon3 human model. AGORA and Recon IDs are very similar so we should be able to match them. We just have to adjust the Recon3 ones a bit. We start by identifying all available exchanges in Recon3 and adjusting the IDs.

In [2]:
from cobra.io import read_sbml_model
import pandas as pd

recon3 = read_sbml_model("../data/Recon3D.xml.gz")
exchanges = pd.Series([r.id for r in recon3.exchanges])
exchanges = exchanges.str.replace("__", "_").str.replace("_e$|EX_", "", regex=True)
exchanges.head()

0     5adtststerone
1    5adtststerones
2             5fthf
3             5htrp
4             5mthf
dtype: object

In [3]:
diet["dilution"] = 1.0
diet.loc[diet.metabolite.isin(exchanges), "dilution"] = 0.2
diet["flux"] = diet["flux"] * diet["dilution"] 
diet[["metabolite", "dilution"]].drop_duplicates().dilution.value_counts()

0.2    79
1.0    12
Name: dilution, dtype: int64

## Adding host supplied components

Finally we add the host metabolites such as primary bile acids and mucins and a minuscule amount of oxygen.

In [4]:
diet.set_index("metabolite", inplace=True)

# mucin
for met in annotations.loc[annotations.metabolite.str.contains("core"), "metabolite"]:
    diet.loc[met, "flux"] = 1

# primary BAs
for met in ["gchola", "tchola"]:
    diet.loc[met, "flux"] = 1

# fiber
diet.loc["cellul", "flux"] = 0.1

# anaerobic
diet.loc["o2", "flux"] = 0.001

diet.reset_index(inplace=True)
diet["reaction"] = "EX_" + diet.metabolite + "(e)"
diet

,metabolite,reaction,flux,dilution
0,etoh,EX_etoh(e),46.886803,0.2
1,h2o,EX_h2o(e),33178.468500,0.2
2,caro,EX_caro(e),0.000717,0.2
3,retinol,EX_retinol(e),0.600450,0.2
4,thm,EX_thm(e),1.081572,0.2
...,...,...,...,...
102,gncore2_rl,EX_gncore2_rl(e),1.000000,NaN
103,core7,EX_core7(e),1.000000,NaN
104,gchola,EX_gchola(e),1.000000,NaN
105,tchola,EX_tchola(e),1.000000,NaN


And we will merge this table with some annotations to make it more accessible.

In [5]:
skeleton = pd.merge(diet, annotations, on="metabolite")

skeleton["global_id"] = skeleton.reaction
skeleton["reaction"] = "EX_" + skeleton.metabolite + "_m"
skeleton.head()

,metabolite,reaction,flux,dilution,name,hmdb,kegg.compound,pubchem.compound,inchi,chebi,global_id
0,etoh,EX_etoh_m,46.886803,0.2,ethanol,HMDB00108,C00469,702.0,"InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",NaN,EX_etoh(e)
1,h2o,EX_h2o_m,33178.468500,0.2,Water,HMDB02111,C00001,962.0,InChI=1S/H2O/h1H2,NaN,EX_h2o(e)
2,caro,EX_caro_m,0.000717,0.2,beta-carotene,HMDB00561,C02094,NaN,NaN,NaN,EX_caro(e)
3,retinol,EX_retinol_m,0.600450,0.2,retinol,HMDB00305,C00473,445354.0,InChI=1S/C20H30O/c1-16(8-6-9-17(2)13-15-21)11-...,NaN,EX_retinol(e)
4,thm,EX_thm_m,1.081572,0.2,Thiamin,HMDB00235,C00378,1130.0,InChI=1S/C12H17N4OS/c1-8-11(3-4-17)18-7-16(8)6...,NaN,EX_thm(e)


## Complete the medium

Great we now have a pretty good skeleton. One issue that this will never be fully complete. There will always be some components missing that are essential for microbial growth. Fortunately, we provide a algorithm in MICOM to complete a medium with the smallest set of additional components to provide growth to all intestinal taxa.

In [6]:
from micom.workflows.db_media import complete_db_medium

manifest, imports = complete_db_medium("../data/agora103_strain.qza", skeleton, growth=0.05, threads=12, max_added_import=10, weights="mass")

Output()

In [7]:
manifest.can_grow.value_counts()

True    818
Name: can_grow, dtype: int64

In [8]:
filled = imports.max()
added = filled[~filled.index.isin(skeleton.reaction)]

print(f"Added flux is {added.sum():.2f}/{filled.sum():.2f} mmol/h.")

Added flux is 27.82/34009.70 mmol/h.


Let's see what was added in large amounts.

In [9]:
added.sort_values(ascending=False)[0:20]

EX_h_m         10.000000
EX_h2_m         5.819827
EX_urea_m       4.227438
EX_for_m        2.407585
EX_glyc_m       2.262650
EX_succ_m       1.152627
EX_gsn_m        0.694061
EX_h2s_m        0.192050
EX_ac_m         0.169726
EX_asn_L_m      0.065890
EX_xyl_D_m      0.059302
EX_ins_m        0.057674
EX_rib_D_m      0.057192
EX_xan_m        0.054624
EX_3mop_m       0.054045
EX_ura_m        0.046962
EX_glyc3p_m     0.044162
EX_n2_m         0.041561
EX_gln_L_m      0.034262
EX_amp_m        0.032259
dtype: float64

Looks okay. So we will now assemble the final medium. For this we add the new components to each sample and rebuild the annotations for a nicely formatted medium.

In [10]:
added_df = added.reset_index() 
added_df.iloc[:, 0] = added_df.iloc[:, 0].str.replace("EX_|_m$", "", regex=True)
added_df.columns = ["metabolite", "flux"]
added_df = pd.concat([skeleton[["metabolite", "flux"]], added_df])

completed = pd.merge(added_df, annotations, on="metabolite", how="left")
completed["reaction"] = "EX_" + completed.metabolite + "_m"
completed["global_id"] = "EX_" + completed.metabolite + "(e)"
completed

,metabolite,flux,name,hmdb,kegg.compound,pubchem.compound,inchi,chebi,reaction,global_id
0,etoh,46.886803,ethanol,HMDB00108,C00469,702.0,"InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",NaN,EX_etoh_m,EX_etoh(e)
1,h2o,33178.468500,Water,HMDB02111,C00001,962.0,InChI=1S/H2O/h1H2,NaN,EX_h2o_m,EX_h2o(e)
2,caro,0.000717,beta-carotene,HMDB00561,C02094,NaN,NaN,NaN,EX_caro_m,EX_caro(e)
3,retinol,0.600450,retinol,HMDB00305,C00473,445354.0,InChI=1S/C20H30O/c1-16(8-6-9-17(2)13-15-21)11-...,NaN,EX_retinol_m,EX_retinol(e)
4,thm,1.081572,Thiamin,HMDB00235,C00378,1130.0,InChI=1S/C12H17N4OS/c1-8-11(3-4-17)18-7-16(8)6...,NaN,EX_thm_m,EX_thm(e)
...,...,...,...,...,...,...,...,...,...,...
160,cmp,0.004205,CMP,HMDB00095,C00055,6131.0,NaN,NaN,EX_cmp_m,EX_cmp(e)
161,coa,0.000310,Coenzyme A,HMDB01423,C00010,6816.0,NaN,NaN,EX_coa_m,EX_coa(e)
162,datp,0.000405,dATP,HMDB01532,C00131,15993.0,NaN,NaN,EX_datp_m,EX_datp(e)
163,fum,0.000155,Fumarate,HMDB00134,C00122,444972.0,"InChI=1S/C4H4O4/c5-3(6)1-2-4(7)8/h1-2H,(H,5,6)...",NaN,EX_fum_m,EX_fum(e)


## Validate the medium

And we will now validate whether the medium works.

In [11]:
from micom.workflows.db_media import check_db_medium

check = check_db_medium("../data/agora103_strain.qza", medium=completed, threads=12)

Output()

In [12]:
check.growth_rate.describe()

count    818.000000
mean       0.070331
std        0.076293
min        0.000277
25%        0.002898
50%        0.047292
75%        0.128204
max        0.323333
Name: growth_rate, dtype: float64

And we are done now and will the save the final medium.

In [13]:
import qiime2 as q2

arti = q2.Artifact.import_data("MicomMedium[Global]", completed)
arti.save("../media/vmh_eu_average_agora.qza")

ModuleNotFoundError: No module named 'qiime2'